In [15]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [16]:
train_df = pd.read_csv('prepared_train1.csv')
train_df = train_df.loc[:, ~train_df.columns.str.contains('^Unnamed')]
train_df

,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,...,TOP_PACK_mixt,TOP_PACK_month,TOP_PACK_net,TOP_PACK_other,TOP_PACK_pilot,TOP_PACK_unlimited,TOP_PACK_week,TOP_PACK_wifi,FREQ_TOP_PACK,user_id
0,4250,15,4251,1417,17,4,388,46,1,1,...,0,0,0,0,0,1,0,0,72,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,95,00000cb4a5d760de88fecb38e2f71b7bec52e834
2,3600,2,1020,340,2,0,90,46,7,0,...,0,0,1,0,0,0,0,0,143,00001654a9d9f96303d9969d0a4a851714a4bb57
3,13500,15,13502,4501,18,43804,41,102,2,0,...,0,0,0,0,0,0,0,0,224,00001dd6fa45f7ba044bd5d84937be464ce78ac2
4,1000,1,985,328,1,0,39,24,0,0,...,0,0,0,0,0,1,0,0,61,000028d9e13a595abe061f9b58f3d76ab907850f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154042,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,95,ffffe85215ddc71a84f95af0afb0deeea90e6967
2154043,6100,15,5800,1933,15,621,26,40,40,0,...,0,0,0,0,0,0,0,0,113,ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15
2154044,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,95,fffff172fda1b4bb38a95385951908bb92379809
2154045,10000,11,7120,2373,13,0,0,140,13,0,...,0,0,1,0,0,0,0,0,228,fffff5911296937a37f09a37a549da2e0dad6dbb


In [17]:
train_df.isna().sum()

MONTANT                 0
FREQUENCE_RECH          0
REVENUE                 0
ARPU_SEGMENT            0
FREQUENCE               0
DATA_VOLUME             0
ON_NET                  0
ORANGE                  0
TIGO                    0
ZONE1                   0
ZONE2                   0
REGULARITY              0
CHURN                   0
TENURE_D 3-6 month      0
TENURE_E 6-9 month      0
TENURE_F 9-12 month     0
TENURE_G 12-15 month    0
TENURE_H 15-18 month    0
TENURE_I 18-21 month    0
TENURE_J 21-24 month    0
TENURE_K > 24 month     0
REGION_DAKAR            0
REGION_DIOURBEL         0
REGION_FATICK           0
REGION_KAFFRINE         0
REGION_KAOLACK          0
REGION_KEDOUGOU         0
REGION_KOLDA            0
REGION_LOUGA            0
REGION_MATAM            0
REGION_OTHER            0
REGION_SAINT-LOUIS      0
REGION_SEDHIOU          0
REGION_TAMBACOUNDA      0
REGION_THIES            0
REGION_ZIGUINCHOR       0
TOP_PACK_daily          0
TOP_PACK_data           0
TOP_PACK_evc

In [18]:
data = train_df[~train_df['CHURN']]
labels = train_df['CHURN']

X_train, X_test, y_train, y_test =train_test_split(data, labels, train_size=0.3, random_state=777)

catboost_model =  CatBoostClassifier()
catboost_model.fit(X_train, y_train)
catboost_model.predict(X_test)

ValueError: Found input variables with inconsistent numbers of samples: [1750061, 2154047]